https://www.bilibili.com/video/BV1AK4y1P7vs?p=1

# 层和块

## 两层的多层感知机

首先，回顾一下多层感知机

In [4]:
import torch
from torch import nn
from torch.nn import functional as F


# 这是一个两层的感知机，
net =nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

# 随机生成输入数据  2是批量大小，现在数据一共就够1个批次，20是数据维度
X=torch.rand(2,20)

net(X)
# 输出就是2*10，因为是两个样本，每个样本输出是10维

tensor([[ 0.0377,  0.1395,  0.2692, -0.2100, -0.0620, -0.2493,  0.2391,  0.0307,
         -0.1823, -0.0657],
        [ 0.1158,  0.1317,  0.2809, -0.2197,  0.0016, -0.2659,  0.1813,  0.1571,
         -0.2954, -0.0216]], grad_fn=<AddmmBackward>)

+ 这里值得注意的是，`nn.Sequential`定义了一种特殊的`Module`。
+ 任何一个神经网络的层都应该是`Module`的子类
+ 不管是一层，还是多层的模型，都应该是`Module`的子类

## 继承Module模块来实现MLP

可以通过继承`Module`来自定义一些层/块
+ 最重要的需要实现的是两个函数，一个就是`__init__`函数，里面放着要定义的层的情况
+ 另一个就是forward函数，这个层的计算过程

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
        
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

```python
dir(F)
help(F)
```
可以用上面的帮助看看，torch.nn.Functional里有很多激活函数

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [8]:
net=MLP()
net(X)

tensor([[-0.0030, -0.1093, -0.0364, -0.1911, -0.0560, -0.1290, -0.0835, -0.1231,
         -0.0680,  0.1367],
        [-0.2054, -0.1313, -0.0451, -0.1426, -0.0807,  0.0965, -0.1160, -0.1647,
         -0.1683,  0.2031]], grad_fn=<AddmmBackward>)

## 顺序块（实现nn.Sequential）

可以用一个函数实现nn.Sequential的功能
+ 其中`*args`表示一个list of arguments，在上面的`nn.Sequential`调用时，传入的参数是一个网络层的列表，所以下面的*args其实也是一个网络层的列表

In [23]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block]=block
            # 这是一个ordered dict 是有序的
    
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X
    
    
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0631, -0.0432,  0.0557, -0.1294, -0.0249,  0.2181,  0.1399, -0.1292,
          0.1484,  0.1344],
        [ 0.1204, -0.0275,  0.1475, -0.1368, -0.0464,  0.1692,  0.1496, -0.2167,
          0.0604,  0.0680]], grad_fn=<AddmmBackward>)

可以看一下关于nn.Module.modules的一些说明，其实作用和上面的那个`self._modules`感觉差不多

In [21]:
help(nn.Module.modules)

Help on function modules in module torch.nn.modules.module:

modules(self) -> Iterator[ForwardRef('Module')]
    Returns an iterator over all modules in the network.
    
    Yields:
        Module: a module in the network
    
    Note:
        Duplicate modules are returned only once. In the following
        example, ``l`` will be returned only once.
    
    Example::
    
        >>> l = nn.Linear(2, 2)
        >>> net = nn.Sequential(l, l)
        >>> for idx, m in enumerate(net.modules()):
                print(idx, '->', m)
    
        0 -> Sequential(
          (0): Linear(in_features=2, out_features=2, bias=True)
          (1): Linear(in_features=2, out_features=2, bias=True)
        )
        1 -> Linear(in_features=2, out_features=2, bias=True)



## 在正向传播函数中执行代码

+ 如果Sequential不能满足你的需求的话，可以继承`nn.Module`来构建自己的网络
+ 主要的好处是在`__init__`和`forward`函数中，可以做大量自定义计算
+ 下面这个例子没有任何意义，只是为了表达，使用继承这种方式自定义的网络，可以进行非常高程度的定制。比如可以让某层的权重不参与训练，类似固定权重，预训练模型，前面的固定，只修改后几层的参数

In [24]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        # 这里注意，这个权重是不参与训练的
        self.linear=nn.Linear(20,20)     
    
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        X=self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()

net=FixedHiddenMLP()
net(X)

tensor(-0.0077, grad_fn=<SumBackward0>)

+ 使用继承的这种方式，可以比使用Sequential方式更好满足自己的需求
+ 反向计算是不需要定义的，只要定义好前向，反向就可以根据前向自动求导

## 混合搭配各种组合块的方法

+ 不管是自己继承`nn.Module`得到的类，还是使用nn.Sequential得到的网络，其实都是nn.Module的子类，所以可以混合着用。
+ 底层是同一套逻辑
+ 网络层可以随便组，随便嵌套，疯狂套娃

In [25]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                              nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
    
    def forward(self,X):
        return self.linear(self.net(X))

chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(-0.3004, grad_fn=<SumBackward0>)

-------------
https://www.bilibili.com/video/BV1AK4y1P7vs?p=2

# 参数管理

+ 假设已经定义好类了/模型结构了，该如何去访问参数
+ 还是以 具有单隐藏层的 多层感知机 为例

In [26]:
import torch
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[-0.2548],
        [-0.2890]], grad_fn=<AddmmBackward>)

## 参数访问

## 把每层的权重都拿出来

In [27]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.0675,  0.2736,  0.0428,  0.1020, -0.1843,  0.0543, -0.1426, -0.1512]])),
             ('bias', tensor([-0.1674]))])

+ 这里net是使用`nn.Sequential`定义的，sequential其实就是类似list的一个东西，可以认为sequential就是python的list
+ 从自动机的角度来讲，每个层的权重就是每个层的状态，是会变的，所以这里访问参数和偏置的时候，调用的是`state_dict`

In [28]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

当然也可以直接访问特定的参数

In [31]:
net[2].weight

Parameter containing:
tensor([[ 0.0675,  0.2736,  0.0428,  0.1020, -0.1843,  0.0543, -0.1426, -0.1512]],
       requires_grad=True)

In [38]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1674], requires_grad=True)
tensor([-0.1674])
None


可以直接调用下面的命令，发现其实有bias和weight属性
```python
dir(net[2])
```

In [40]:
net[2].weight.grad==None

# 由于这里还没有进行反向计算，所以grad为None

True

## 也可以一次性访问所有参数

In [41]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


+ 这里就会给出参数的名称，和对应参数的形状。
+ 如果在定义这些参数的时候，没有显示指定名称，则就默认 `#num.weight/bias`，层的名称加上是权重还是偏置，就可以了
+ ReLU，也就是序号为1的层，没有权重，所以没有参数，取不出来东西
+ 另外，有了名字就可以用名字来获取参数的信息

In [42]:
net.state_dict()["2.bias"].data

tensor([-0.1674])

In [43]:
print([(name,param.shape) for name,param in net[0].named_parameters()])
print([(name,param.shape) for name,param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


可以看到，这部分代码没有前面的`*`号也可以运行，只是输出结果会是列表，而不是上面的元组。

## 从嵌套块收集参数

In [49]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
    net =nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
        # 第一个参数是name，第二个参数是网络层module
    return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.6426],
        [0.6423]], grad_fn=<AddmmBackward>)

In [50]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

+ 可以看到，嵌套时候的网络表示，稍微复杂了一点，也嵌套了
+ 这个时候如果想从嵌套块收集参数，就要像json嵌套访问一样，一层一层去看
+ 最外层是 0→block2,1→linear
+ 0之后就是block2
+ 注意，这里只支持数字访问，不支持module的name的str访问

In [60]:
print(rgnet[0][0])
print(rgnet[0][0][0])

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
)
Linear(in_features=4, out_features=8, bias=True)


+ 模型比较复杂的时候，看起来就会比较费劲
+ 所以需要设计好模块化的东西，这样对自己会比较方便

In [59]:
net.state_dict()["2.bias"].data

tensor([-0.1674])

7:50